In [6]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re
import os
from datetime import datetime

# --------------------------
# List of tickers
# --------------------------
tickers = [
   "1JANATAMF", "1STPRIMFMF", "AAMRANET", "AAMRATECH", "ABB1STMF", "ABBANK", "ACFL", "ACI", "ACIFORMULA", "ACMELAB",
    "ACMEPL", "ACTIVEFINE", "ADNTEL", "ADVENT", "AFCAGRO", "AFTABAUTO", "AGNISYSL", "AGRANINS", "AIBL1STIMF", "AIL",
    "AL-HAJTEX", "ALARABANK", "ALIF", "ALLTEX", "AMANFEED", "AMBEEPHA", "AMCL(PRAN)", "ANLIMAYARN", "ANWARGALV", "AOL",
    "APEXFOODS", "APEXFOOT", "APEXSPINN", "APEXTANRY", "APOLOISPAT", "ARAMIT", "ARAMITCEM", "ARGONDENIM", "ASIAINS",
    "ASIAPACINS", "ASIATICLAB", "ATCSLGF", "ATLASBANG", "AZIZPIPES", "BANGAS", "BANKASIA", "BARKAPOWER", "BATASHOE",
    "BATBC", "BAYLEASING", "BBS", "BBSCABLES", "BDAUTOCA", "BDCOM", "BDFINANCE", "BDLAMPS", "BDSERVICE", "BDTHAI",
    "BDTHAIFOOD", "BDWELDING", "BEACHHATCH", "BEACONPHAR", "BENGALWTL", "BERGERPBL", "BESTHLDNG", "BEXIMCO", "BGIC",
    "BIFC", "BNICL", "BPML", "BPPL", "BRACBANK", "BSC", "BSCPLC", "BSRMLTD", "BSRMSTEEL", "BXPHARMA", "CAPITECGBF",
    "CAPMBDBLMF", "CAPMIBBLMF", "CENTRALINS", "CENTRALPHL", "CITYBANK", "CITYGENINS", "CLICL", "CNATEX", "CONFIDCEM",
    "CONTININS", "COPPERTECH", "CROWNCEMNT", "CRYSTALINS", "CVOPRL", "DACCADYE", "DAFODILCOM", "DBH", "DBH1STMF",
    "DELTALIFE", "DELTASPINN", "DESCO", "DESHBANDHU", "DGIC", "DHAKABANK", "DHAKAINS", "DOMINAGE", "DOREENPWR",
    "DSHGARME", "DSSL", "DULAMIACOT", "DUTCHBANGL", "EASTERNINS", "EASTLAND", "EASTRNLUB", "EBL", "EBL1STMF",
    "EBLNRBMF", "ECABLES", "EGEN", "EHL", "EIL", "EMERALDOIL", "ENVOYTEX", "EPGL", "ESQUIRENIT", "ETL", "EXIM1STMF",
    "EXIMBANK", "FAMILYTEX", "FARCHEM", "FAREASTFIN", "FAREASTLIF", "FASFIN", "FBFIF", "FEDERALINS", "FEKDIL",
    "FINEFOODS", "FIRSTFIN", "FIRSTSBANK", "FORTUNE", "FUWANGCER", "FUWANGFOOD", "GBBPOWER", "GEMINISEA", "GENEXIL",
    "GENNEXT", "GHAIL", "GHCL", "GIB", "GLDNJMF", "GLOBALINS", "GOLDENSON", "GP", "GPHISPAT", "GQBALLPEN", "GRAMEENS2",
    "GREENDELMF", "GREENDELT", "GSPFINANCE", "HAKKANIPUL", "HAMI", "HEIDELBCEM", "HFL", "HRTEX", "HWAWELLTEX", "IBNSINA",
    "IBP", "ICB", "ICB3RDNRB", "ICBAGRANI1", "ICBAMCL2ND", "ICBEPMF1S1", "ICBIBANK", "ICBSONALI1", "ICICL", "IDLC",
    "IFADAUTOS", "IFIC", "IFIC1STMF", "IFILISLMF1", "ILFSL", "INDEXAGRO", "INTECH", "INTRACO", "IPDC", "ISLAMIBANK",
    "ISLAMICFIN", "ISLAMIINS", "ISNLTD", "ITC", "JAMUNABANK", "JAMUNAOIL", "JANATAINS", "JHRML", "JMISMDL", "JUTESPINN",
    "KARNAPHULI", "KAY&QUE", "KBPPWBIL", "KDSALTD", "KEYACOSMET", "KOHINOOR", "KPCL", "KPPL", "KTL", "LANKABAFIN",
    "LEGACYFOOT", "LHB", "LIBRAINFU", "LINDEBD", "LOVELLO", "LRBDL", "LRGLOBMF1", "MAGURAPLEX", "MAKSONSPIN", "MALEKSPIN",
    "MARICO", "MATINSPINN", "MBL1STMF", "MEGCONMILK", "MEGHNACEM", "MEGHNAINS", "MEGHNALIFE", "MEGHNAPET", "MERCANBANK",
    "MERCINS", "METROSPIN", "MHSML", "MIDASFIN", "MIDLANDBNK", "MIRACLEIND", "MIRAKHTER", "MITHUNKNIT", "MJLBD", "MLDYEING",
    "MONNOAGML", "MONNOCERA", "MONNOFABR", "MONOSPOOL", "MPETROLEUM", "MTB", "NAHEEACP", "NATLIFEINS", "NAVANACNG",
    "NAVANAPHAR", "NBL", "NCCBANK", "NCCBLMF1", "NEWLINE", "NFML", "NHFIL", "NITOLINS", "NORTHERN", "NORTHRNINS",
    "NPOLYMER", "NRBBANK", "NRBCBANK", "NTC", "NTLTUBES", "NURANI", "OAL", "OIMEX", "OLYMPIC", "ONEBANKPLC", "ORIONINFU",
    "ORIONPHARM", "PADMALIFE", "PADMAOIL", "PARAMOUNT", "PDL", "PENINSULA", "PEOPLESINS", "PF1STMF", "PHARMAID",
    "PHENIXINS", "PHOENIXFIN", "PHPMF1", "PIONEERINS", "PLFSL", "POPULAR1MF", "POPULARLIF", "POWERGRID", "PRAGATIINS",
    "PRAGATILIF", "PREMIERBAN", "PREMIERCEM", "PREMIERLEA", "PRIME1ICBA", "PRIMEBANK", "PRIMEFIN", "PRIMEINSUR",
    "PRIMELIFE", "PRIMETEX", "PROGRESLIF", "PROVATIINS", "PTL", "PUBALIBANK", "PURABIGEN", "QUASEMIND", "QUEENSOUTH",
    "RAHIMAFOOD", "RAHIMTEXT", "RAKCERAMIC", "RANFOUNDRY", "RDFOOD", "RECKITTBEN", "REGENTTEX", "RELIANCE1", "RELIANCINS",
    "RENATA", "RENWICKJA", "REPUBLIC", "RINGSHINE", "ROBI", "RSRMSTEEL", "RUNNERAUTO", "RUPALIBANK", "RUPALIINS",
    "RUPALILIFE", "SAFKOSPINN", "SAIFPOWER", "SAIHAMCOT", "SAIHAMTEX", "SALAMCRST", "SALVOCHEM", "SAMATALETH", "SAMORITA",
    "SANDHANINS", "SAPORTL", "SAVAREFR", "SBACBANK", "SEAPEARL", "SEMLFBSLGF", "SEMLIBBLSF", "SEMLLECMF", "SHAHJABANK",
    "SHARPIND", "SHASHADNIM", "SHEPHERD", "SHURWID", "SHYAMPSUG", "SIBL", "SICL", "SILCOPHL", "SILVAPHL", "SIMTEX",
    "SINGERBD", "SINOBANGLA", "SIPLC", "SKTRIMS", "SONALIANSH", "SONALILIFE", "SONALIPAPR", "SONARBAINS", "SONARGAON",
    "SOUTHEASTB", "SPCERAMICS", "SPCL", "SQUARETEXT", "SQURPHARMA", "SSSTEEL", "STANCERAM", "STANDARINS", "STANDBANKL",
    "STYLECRAFT", "SUMITPOWER", "SUNLIFEINS", "TAKAFULINS", "TALLUSPIN", "TAMIJTEX", "TECHNODRUG", "TILIL", "TITASGAS",
    "TOSRIFA", "TRUSTB1MF", "TRUSTBANK", "TUNGHAI", "UCB", "UNILEVERCL", "UNIONBANK", "UNIONCAP", "UNIONINS", "UNIQUEHRL",
    "UNITEDFIN", "UNITEDINS", "UPGDCL", "USMANIAGL", "UTTARABANK", "UTTARAFIN", "VAMLBDMF1", "VAMLRBBF", "VFSTDL",
    "WALTONHIL", "WATACHEM", "WMSHIPYARD", "YPL", "ZAHEENSPIN", "ZAHINTEX", "ZEALBANGLA"
]

# Regex pattern to extract shareholding numbers
pattern = r"(Sponsor/Director|Govt|Institute|Foreign|Public):\s*([\d\.]+)"

# Initialize list to store all rows
all_data = []

headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"
}

for ticker in tickers:
    url = f"https://dsebd.org/displayCompany.php?name={ticker}"
    print(f"Processing: {ticker}")

    try:
        response = requests.get(url, headers=headers, timeout=10)
        response.raise_for_status()
        soup = BeautifulSoup(response.text, "html.parser")
    except requests.exceptions.RequestException as e:
        print(f"Failed to fetch {ticker}: {e}")
        continue
    except Exception as e:
        print(f"Error parsing {ticker}: {e}")
        continue
    
    # Find table containing "Share Holding Percentage"
    target_table = None
    for table in soup.find_all("table"):
        if "Share Holding Percentage" in table.get_text():
            target_table = table
            break
    
    if not target_table:
        print(f"Shareholding table not found for {ticker}")
        continue
    
    rows = target_table.find_all("tr")
    i = 0
    while i < len(rows):
        tr = rows[i]
        tds = [td.get_text(" ", strip=True) for td in tr.find_all("td")]

        if any("Share Holding Percentage" in td for td in tds):
            # Extract date from this row or next rows
            date_str = None
            
            # Try to extract date from current row first
            for td in tds:
                # Match pattern like: "as on Jun 30, 2024"
                match = re.search(r"[Aa]s\s+on\s+([A-Za-z]{3}\s+\d{1,2},\s*\d{4})", td)
                if match:
                    date_str = match.group(1).strip()
                    break
            
            # If not found, look in next few rows
            if not date_str:
                j = i + 1
                while j < min(i + 3, len(rows)):  # Look ahead 2-3 rows max
                    next_tds = [td.get_text(" ", strip=True) for td in rows[j].find_all("td")]
                    for td in next_tds:
                        match = re.search(r"[Aa]s\s+on\s+([A-Za-z]{3}\s+\d{1,2},\s*\d{4})", td)
                        if match:
                            date_str = match.group(1).strip()
                            break
                    if date_str:
                        break
                    j += 1
            
            # Parse date to dd.mm.yyyy
            parsed_date = None
            if date_str:
                try:
                    # Convert "Jun 30, 2024" to datetime
                    dt = datetime.strptime(date_str, "%b %d, %Y")
                    parsed_date = dt.strftime("%d.%m.%Y")  # Format: dd.mm.yyyy
                except ValueError:
                    print(f"Date format error for {ticker}: '{date_str}'")
                    parsed_date = None
            
            # Combine subsequent rows for shareholding data
            j = i + 1
            combined_text = ""
            while j < len(rows):
                next_tds = [td.get_text(" ", strip=True) for td in rows[j].find_all("td")]
                if any("Share Holding Percentage" in td for td in next_tds):
                    break
                combined_text += " ".join(next_tds) + " "
                j += 1
            
            matches = re.findall(pattern, combined_text)
            if matches:
                row_dict = {"Ticker": ticker, "Date": parsed_date}
                for cat, val in matches:
                    row_dict[cat] = float(val)
                
                # Add missing categories with None
                for cat in ["Sponsor/Director", "Govt", "Institute", "Foreign", "Public"]:
                    if cat not in row_dict:
                        row_dict[cat] = None
                        
                all_data.append(row_dict)
            else:
                print(f"No matches found in shareholding data for {ticker}")
            
            i = j
        else:
            i += 1

# Create DataFrame
df = pd.DataFrame(all_data)

# Ensure all expected columns exist
expected_cols = ["Ticker", "Date", "Sponsor/Director", "Govt", "Institute", "Foreign", "Public"]
for col in expected_cols:
    if col not in df.columns:
        df[col] = None

# Reorder columns
df = df[expected_cols]

# --------------------------
# Save to Desktop
# --------------------------
desktop = os.path.join(os.path.expanduser("~"), "Desktop")
output_path = os.path.join(desktop, "dse_shareholding_data.xlsx")

# Handle potential file permission errors
try:
    df.to_excel(output_path, index=False)
    print_path = output_path
except PermissionError:
    print("File is open or permission denied. Saving with a new filename...")
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    output_path = os.path.join(desktop, f"dse_shareholding_data_{timestamp}.xlsx")
    df.to_excel(output_path, index=False)
    print_path = output_path

print("\n✅ Data scraping completed successfully!")
print(f"📌 File saved at: {print_path}")
print(f"📊 Total records: {len(df)}")
if not df.empty:
    print(df.tail())

Processing: 1JANATAMF
Processing: 1STPRIMFMF
Processing: AAMRANET
Processing: AAMRATECH
Processing: ABB1STMF
Processing: ABBANK
Processing: ACFL
Processing: ACI
Processing: ACIFORMULA
Processing: ACMELAB

✅ Data scraping completed successfully!
📌 File saved at: C:\Users\hp\Desktop\dse_shareholding_data.xlsx
📊 Total records: 30
        Ticker        Date  Sponsor/Director  Govt  Institute  Foreign  Public
25  ACIFORMULA  31.08.2025             66.02   0.0      22.49     0.00   11.49
26  ACIFORMULA  30.09.2025             66.02   0.0      21.62     0.00   12.36
27     ACMELAB  30.06.2024             39.88   0.0      31.16     0.14   28.82
28     ACMELAB  31.08.2025             42.38   0.0      29.64     0.00   27.98
29     ACMELAB  30.09.2025             42.38   0.0      29.96     0.00   27.66
